In [1]:
import tensorflow as tf
import numpy as np
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [2]:
import numpy as np


a = tf.constant(2, tf.int16)
b = tf.constant(4, tf.float32)
c = tf.constant(8, tf.float32)

d = tf.Variable(2, tf.int16)
e = tf.Variable(4, tf.float32)
f = tf.Variable(8, tf.float32)

g = tf.constant(np.zeros(shape=(2,2), dtype=np.float32)) #可以正常声明变量

h = tf.zeros([11], tf.int16)
i = tf.ones([2,2], tf.float32)
j = tf.zeros([1000,4,3], tf.float64)

k = tf.Variable(tf.zeros([2,2], tf.float32))
l = tf.Variable(tf.zeros([5,6,5], tf.float32))

In [11]:
a = tf.constant(2, tf.int16)
b = tf.constant(4, tf.float32)

graph = tf.Graph()
with graph.as_default():
    a = tf.Variable(8, tf.float32)
    b = tf.Variable(tf.zeros([2,2], tf.float32))
    
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print(f)
    print(session.run(a))
    print(session.run(b))


<tf.Variable 'Variable_2:0' shape=() dtype=int32_ref>
8
[[ 0.  0.]
 [ 0.  0.]]


In [3]:
a=tf.constant([1,2],name="a")
b=tf.constant([2,4],name="b")
result = a+b
print(result)

Tensor("add:0", shape=(2,), dtype=int32)


In [6]:
a=tf.constant([1,2,3,4])
b=tf.constant([1,2,3,4])
result=a+b
sess=tf.Session()
print(sess.run(result))
sess.close

#输出 [2 4 6 8]

with tf.Session() as sess:
    a=tf.constant([1,2,3,4])
    b=tf.constant([1,2,3,4])
    result=a+b
    print(sess.run(result))
    
#输出 [2 4 6 8]

[2 4 6 8]
[2 4 6 8]


In [16]:
w1=tf.Variable(tf.random_normal([1,2],stddev=1,seed=1))

#因为需要重复输入x，而每建一个x就会生成一个结点，计算图的效率会低。所以使用占位符
x=tf.placeholder(tf.float32,shape=(1,2))
x1=tf.constant([[0.7,0.9]])

a=x+w1
b=x1+w1

sess=tf.Session()
sess.run(tf.global_variables_initializer())
#运行y时将占位符填上，feed_dict为字典，变量名不可变
y_1=sess.run(a,feed_dict={x:[[0.7,0.9]]})
y_2=sess.run(b)
print(y_1)
print(y_2)
sess.close

[[-0.11131823  2.38459873]]
[[-0.11131823  2.38459873]]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x000001D9DB209D30>>

In [13]:
list_of_points1_ = [[1,2], [3,4], [5,6], [7,8]]
list_of_points2_ = [[15,16], [13,14], [11,12], [9,10]]
list_of_points1 = np.array([np.array(elem).reshape(1,2) for elem in list_of_points1_])
list_of_points2 = np.array([np.array(elem).reshape(1,2) for elem in list_of_points2_])

graph = tf.Graph()
with graph.as_default():   
    
    #我们使用 tf.placeholder() 创建占位符 ，在 session.run() 过程中再投递数据 
    point1 = tf.placeholder(tf.float32, shape=(1, 2))
    point2 = tf.placeholder(tf.float32, shape=(1, 2))
    
    def calculate_eucledian_distance(point1, point2):
        difference = tf.subtract(point1, point2)
        power2 = tf.pow(difference, tf.constant(2.0, shape=(1,2)))
        add = tf.reduce_sum(power2)
        eucledian_distance = tf.sqrt(add)
        return eucledian_distance
    
    dist = calculate_eucledian_distance(point1, point2)
    
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()   
    for ii in range(len(list_of_points1)):
        point1_ = list_of_points1[ii]
        point2_ = list_of_points2[ii]
        
        #使用feed_dict将数据投入到[dist]中
        feed_dict = {point1 : point1_, point2 : point2_}
        distance = session.run([dist], feed_dict=feed_dict)
        print("the distance between {} and {} -> {}".format(point1_, point2_, distance))

the distance between [[1 2]] and [[15 16]] -> [19.79899]
the distance between [[3 4]] and [[13 14]] -> [14.142136]
the distance between [[5 6]] and [[11 12]] -> [8.485281]
the distance between [[7 8]] and [[ 9 10]] -> [2.8284271]


In [8]:
import tensorflow as tf
from numpy.random import RandomState

batch_size=10
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))


# None 可以根据batch 大小确定维度，在shape的一个维度上使用None，方便不大的batch
x=tf.placeholder(tf.float32,shape=(None,2))
y=tf.placeholder(tf.float32,shape=(None,1))

a=tf.nn.relu(tf.matmul(x,w1))
yhat=tf.nn.relu(tf.matmul(a,w2))

#定义交叉熵为损失函数，训练过程使用Adam算法最小化交叉熵
cross_entropy=-tf.reduce_mean(y*tf.log(tf.clip_by_value(yhat,1e-10,1.0)))
train_step=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm=RandomState(1)
data_size=512

#生成两个特征，共data_size个样本
X=rdm.rand(data_size,2)
#定义规则给出样本标签，所有x1+x2<1的样本认为是正样本，其他为负样本。Y，1为正样本
Y = [[int(x1+x2 < 1)] for (x1, x2) in X]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w1))
    print(sess.run(w2))
    steps=15001
    for i in range(steps):
        
        #选定每一个批量读取的首尾位置，确保在1个epoch内采样训练
        start = i * batch_size % data_size
        end = min(start + batch_size,data_size)
        sess.run(train_step,feed_dict={x:X[start:end],y:Y[start:end]})
        if i % 1000 == 0:
            training_loss= sess.run(cross_entropy,feed_dict={x:X,y:Y})
            print("在迭代 %d 次后，训练损失为 %g"%(i,training_loss))
        if i == steps-1:
            print(sess.run(w1))
            print(sess.run(w2))#输出更新后的权重矩阵

[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
在迭代 0 次后，训练损失为 0.308504
在迭代 1000 次后，训练损失为 0.0393406
在迭代 2000 次后，训练损失为 0.0182158
在迭代 3000 次后，训练损失为 0.0104779
在迭代 4000 次后，训练损失为 0.00680374
在迭代 5000 次后，训练损失为 0.00446512
在迭代 6000 次后，训练损失为 0.00296797
在迭代 7000 次后，训练损失为 0.00218553
在迭代 8000 次后，训练损失为 0.00179452
在迭代 9000 次后，训练损失为 0.0013211
在迭代 10000 次后，训练损失为 0.000957699
在迭代 11000 次后，训练损失为 0.00081103
在迭代 12000 次后，训练损失为 0.000643147
在迭代 13000 次后，训练损失为 0.00047439
在迭代 14000 次后，训练损失为 0.00030086
在迭代 15000 次后，训练损失为 0.000137936
[[-0.81131822  3.84255528  3.38165283]
 [-2.4427042   1.98635983  3.50722313]]
[[-0.81131822]
 [ 4.02907705]
 [ 2.60285187]]


上面的代码定义了一个简单的三层全连接网络（输入层、隐藏层和输出层分别为2、3和2个神经元），隐藏层和输出层的激活函数使用的是ReLU函数。该模型训练的样本总数为512，每次迭代读取的批量为10。这个简单的全连接网络以交叉熵为损失函数，并使用Adam优化算法进行权重更新。

其中需要注意的几个函数如tf.nn.relu()代表调用ReLU激活函数，tf.matmul()为矩阵乘法等。tf.clip_by_value(yhat,1e-10,1.0)这一语句代表的是截断yhat的值，因为这一语句是嵌套在tf.log()函数内的，所以我们需要确保yhat的取值不会导致对数无穷大。

tf.train.AdamOptimizer(learning_rate).minimize(cost_function)是进行训练的函数，其中我们采用的是Adam优化算法更新权重，并且需要提供学习速率和损失函数这两个参数。后面就是生成训练数据，X=rdm.rand(512,2)表示随机生成512个样本，每个样本有两个特征值。最后就是迭代运行了，这里我们计算出每一次迭代抽取数据的起始位置（start）和结束位置（end），并且每一次抽取的数据量为前面我们定义的批量，如果一个epoch最后剩余的数据少于批量大小，那就只是用剩余的数据进行训练。最后两句代码是为了计算训练损失并迭代一些次数后输出训练损失。

In [3]:
import numpy as np
import pickle
import json
import os

#定义一些预处理函数

def flatten_tf_array(array):
    shape = array.get_shape().as_list()
    return tf.reshape(array, [shape[0], shape[1] * shape[2] * shape[3]])

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

def one_hot_encode(np_array):
    return (np.arange(10) == np_array[:,None]).astype(np.float32)

def reformat_data(dataset, labels, image_width, image_height, image_depth):
    np_dataset_ = np.array([np.array(image_data).reshape(image_width, image_height, image_depth) for image_data in dataset])
    np_labels_ = one_hot_encode(np.array(labels, dtype=np.float32))
    np_dataset, np_labels = randomize(np_dataset_, np_labels_)
    return np_dataset, np_labels

In [4]:
cifar10_folder = './data/cifar10/'
train_datasets = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', ]
test_dataset = ['test_batch']
c10_image_height = 32
c10_image_width = 32
c10_image_depth = 3
c10_num_labels = 10
c10_image_size = 32 #Ahmet Taspinar的代码缺少了这一语句

with open(cifar10_folder + test_dataset[0], 'rb') as f0:
    c10_test_dict = pickle.load(f0, encoding='bytes')

c10_test_dataset, c10_test_labels = c10_test_dict[b'data'], c10_test_dict[b'labels']
test_dataset_cifar10, test_labels_cifar10 = reformat_data(c10_test_dataset, c10_test_labels, c10_image_size, c10_image_size, c10_image_depth)

c10_train_dataset, c10_train_labels = [], []
for train_dataset in train_datasets:
    with open(cifar10_folder + train_dataset, 'rb') as f0:
        c10_train_dict = pickle.load(f0, encoding='bytes')
        c10_train_dataset_, c10_train_labels_ = c10_train_dict[b'data'], c10_train_dict[b'labels']
 
        c10_train_dataset.append(c10_train_dataset_)
        c10_train_labels += c10_train_labels_

c10_train_dataset = np.concatenate(c10_train_dataset, axis=0)
train_dataset_cifar10, train_labels_cifar10 = reformat_data(c10_train_dataset, c10_train_labels, c10_image_size, c10_image_size, c10_image_depth)
del c10_train_dataset
del c10_train_labels

print("训练集包含以下标签: {}".format(np.unique(c10_train_dict[b'labels'])))
print('训练集维度', train_dataset_cifar10.shape, train_labels_cifar10.shape)
print('测试集维度', test_dataset_cifar10.shape, test_labels_cifar10.shape)


训练集包含以下标签: [0 1 2 3 4 5 6 7 8 9]
训练集维度 (50000, 32, 32, 3) (50000, 10)
测试集维度 (10000, 32, 32, 3) (10000, 10)


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST/", one_hot=True)

Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz


In [5]:
print("Training data size: ", mnist.train.num_examples) 
print ("Validating data size: ", mnist.validation.num_examples) 
print ("Testing data size: ", mnist.test.num_examples) 

Training data size:  55000
Validating data size:  5000
Testing data size:  10000


In [10]:
print(mnist.train.images[0]) 
print(mnist.train.labels[0]) 
print(mnist.test.labels[0]) 
print(mnist.validation.labels[0]) 

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST/", one_hot=True)


INPUT_NODE = 784     
OUTPUT_NODE = 10     
LAYER1_NODE = 500         
                              
BATCH_SIZE = 100        

# 模型相关的参数
LEARNING_RATE_BASE = 0.8      
LEARNING_RATE_DECAY = 0.99    
REGULARAZTION_RATE = 0.0001   
TRAINING_STEPS = 10000        
MOVING_AVERAGE_DECAY = 0.99 

def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 使用滑动平均类
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights2) + biases2

    else:
        
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)  
    
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义训练轮数及相关的滑动平均类 
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    regularaztion = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularaztion
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化回话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})

        test_acc=sess.run(accuracy,feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" %(TRAINING_STEPS, test_acc)))


Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz


In [8]:
train(mnist)

After 0 training step(s), validation accuracy using average model is 0.0912 
After 1000 training step(s), validation accuracy using average model is 0.9756 
After 2000 training step(s), validation accuracy using average model is 0.9794 
After 3000 training step(s), validation accuracy using average model is 0.9818 
After 4000 training step(s), validation accuracy using average model is 0.9814 
After 5000 training step(s), test accuracy using average model is 0.9834


In [7]:
avg_class = None
train(mnist)

After 0 training step(s), validation accuracy using average model is 0.1678 
After 1000 training step(s), validation accuracy using average model is 0.9766 
After 2000 training step(s), validation accuracy using average model is 0.981 
After 3000 training step(s), validation accuracy using average model is 0.9808 
After 4000 training step(s), validation accuracy using average model is 0.9824 
After 5000 training step(s), validation accuracy using average model is 0.9822 
After 6000 training step(s), validation accuracy using average model is 0.983 
After 7000 training step(s), validation accuracy using average model is 0.9828 
After 8000 training step(s), validation accuracy using average model is 0.9842 
After 9000 training step(s), validation accuracy using average model is 0.9834 
After 10000 training step(s), test accuracy using average model is 0.9851


In [32]:
LENET5_LIKE_BATCH_SIZE = 32
LENET5_LIKE_FILTER_SIZE = 5
LENET5_LIKE_FILTER_DEPTH = 16
LENET5_LIKE_NUM_HIDDEN = 120

def variables_lenet5_like(filter_size = LENET5_LIKE_FILTER_SIZE, 
                          filter_depth = LENET5_LIKE_FILTER_DEPTH, 
                          num_hidden = LENET5_LIKE_NUM_HIDDEN,
                          image_width = 32, image_height = 32, image_depth = 3, num_labels = 10):
    
    w1 = tf.Variable(tf.truncated_normal([filter_size, filter_size, image_depth, filter_depth], stddev=0.1))
    b1 = tf.Variable(tf.zeros([filter_depth]))

    w2 = tf.Variable(tf.truncated_normal([filter_size, filter_size, filter_depth, filter_depth], stddev=0.1))
    b2 = tf.Variable(tf.constant(1.0, shape=[filter_depth]))
   
    w3 = tf.Variable(tf.truncated_normal([(image_width // 4)*(image_height // 4)*filter_depth , num_hidden], stddev=0.1))
    b3 = tf.Variable(tf.constant(1.0, shape = [num_hidden]))

    w4 = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    b4 = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    
    w5 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    b5 = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    variables = {
        'w1': w1, 'w2': w2, 'w3': w3, 'w4': w4, 'w5': w5,
        'b1': b1, 'b2': b2, 'b3': b3, 'b4': b4, 'b5': b5
    }
    return variables

def model_lenet5_like(data, variables):
    layer1_conv = tf.nn.conv2d(data, variables['w1'], [1, 1, 1, 1], padding='SAME')
    layer1_actv = tf.nn.relu(layer1_conv + variables['b1'])
    layer1_pool = tf.nn.avg_pool(layer1_actv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

    layer2_conv = tf.nn.conv2d(layer1_pool, variables['w2'], [1, 1, 1, 1], padding='SAME')
    layer2_actv = tf.nn.relu(layer2_conv + variables['b2'])
    layer2_pool = tf.nn.avg_pool(layer2_actv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    flat_layer = flatten_tf_array(layer2_pool)
    layer3_fccd = tf.matmul(flat_layer, variables['w3']) + variables['b3']
    layer3_actv = tf.nn.relu(layer3_fccd)
    layer3_drop = tf.nn.dropout(layer3_actv, 0.5)
    
    layer4_fccd = tf.matmul(layer3_actv, variables['w4']) + variables['b4']
    layer4_actv = tf.nn.relu(layer4_fccd)
    layer4_drop = tf.nn.dropout(layer4_actv, 0.5)
    
    logits = tf.matmul(layer4_actv, variables['w5']) + variables['b5']
    return logits


#Variables used in the constructing and running the graph
num_steps = 10001
display_step = 1000
learning_rate = 0.001
batch_size = 16

#定义数据的基本信息，传入变量
image_width = 32
image_height = 32
image_depth = 3
num_labels = 10


test_dataset = test_dataset_cifar10
test_labels = test_labels_cifar10
train_dataset = train_dataset_cifar10
train_labels = train_labels_cifar10




graph = tf.Graph()
with graph.as_default():
    #1 首先使用占位符定义数据变量的维度
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_width, image_height, image_depth))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_test_dataset = tf.constant(test_dataset, tf.float32)

    #2 然后初始化权重矩阵和偏置向量
    variables = variables_lenet5_like(image_width = image_width, image_height=image_height, image_depth = image_depth, num_labels = num_labels)


    #3 使用模型计算分类
    logits = model_lenet5_like(tf_train_dataset, variables)

    #4 使用带softmax的交叉熵函数计算预测标签和真实标签之间的损失函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

    #5  采用Adam优化算法优化上一步定义的损失函数，给定学习率
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    # 执行预测推断
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(model_lenet5_like(tf_test_dataset, variables))


with tf.Session(graph=graph) as session:
    #初始化全部变量
    tf.global_variables_initializer().run()
    print('Initialized with learning_rate', learning_rate)
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        #在每一次批量中，获取当前的训练数据，并传入feed_dict以馈送到占位符中
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        train_accuracy = accuracy(predictions, batch_labels)
        
        if step % display_step == 0:
            test_accuracy = accuracy(test_prediction.eval(), test_labels)
            message = "step {:04d} : loss is {:06.2f}, accuracy on training set {:02.2f} %, accuracy on test set {:02.2f} %".format(step, l, train_accuracy, test_accuracy)
            print(message)

Initialized with learning_rate 0.001
step 0000 : loss is 057.10, accuracy on training set 6.25 %, accuracy on test set 9.34 %
step 1000 : loss is 001.63, accuracy on training set 62.50 %, accuracy on test set 34.26 %
step 2000 : loss is 001.38, accuracy on training set 50.00 %, accuracy on test set 40.97 %
step 3000 : loss is 001.78, accuracy on training set 31.25 %, accuracy on test set 41.43 %
step 4000 : loss is 001.41, accuracy on training set 56.25 %, accuracy on test set 42.54 %
step 5000 : loss is 001.52, accuracy on training set 37.50 %, accuracy on test set 45.20 %
step 6000 : loss is 001.28, accuracy on training set 56.25 %, accuracy on test set 42.72 %
step 7000 : loss is 001.29, accuracy on training set 56.25 %, accuracy on test set 46.89 %
step 8000 : loss is 001.43, accuracy on training set 43.75 %, accuracy on test set 47.97 %
step 9000 : loss is 001.58, accuracy on training set 31.25 %, accuracy on test set 49.00 %
step 10000 : loss is 001.41, accuracy on training set 4